In [2]:
import os
import joblib
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# --- Configuración inicial ---
MODEL_DIR = "./models"  # carpeta donde están los .joblib
OUTPUT_DIR = "./evaluation_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Carga de datasets ---
# A
y_A_train = pd.read_csv("./A_sets/y_train_inicial.csv")
y_A_test  = pd.read_csv("./A_sets/y_test_inicial.csv")
X_A_train = pd.read_csv("./A_sets/x_A_train_std.csv")
X_A_test  = pd.read_csv("./A_sets/x_A_test_std.csv")

# B
y_B_train = pd.read_csv("./B_sets/y_train_inicial.csv")
y_B_test  = pd.read_csv("./B_sets/y_test_inicial.csv")
X_B_train = pd.read_csv("./B_sets/x_B_train_std.csv")
X_B_test  = pd.read_csv("./B_sets/x_B_test_std.csv")

# --- Limpieza de datasets ---
def drop_index_col(df):
    if "Unnamed: 0" in df.columns:
        return df.drop(columns=["Unnamed: 0"])
    return df

X_A_train = drop_index_col(X_A_train)
X_A_test  = drop_index_col(X_A_test)
X_B_train = drop_index_col(X_B_train)
X_B_test  = drop_index_col(X_B_test)

# Convertir y a Series
def y_to_series(y_df):
    if isinstance(y_df, pd.DataFrame):
        if y_df.shape[1] == 1:
            return y_df.iloc[:,0].squeeze()
        else:
            raise ValueError("y tiene más de una columna; ajustar selección.")
    return y_df

y_A_train = y_to_series(y_A_train)
y_A_test  = y_to_series(y_A_test)
y_B_train = y_to_series(y_B_train)
y_B_test  = y_to_series(y_B_test)

# --- Modelos a evaluar ---
model_files = {
    "LASSO_A_std": "LASSO_A_std.joblib",
    "Neural_Network_A_std": "Neural_Network_A_std.joblib",
    "RandomForest_B_std": "RandomForest_B_std.joblib",
    "Regresor_Polinomico_A_std": "Regresor_Polinomico_A_std.joblib",
    "XGBoost_A_std": "XGBoost_A_std.joblib"
}

# --- Funciones auxiliares ---
def align_features_for_model(X, model):
    Xc = X.copy()
    if hasattr(model, "feature_names_in_"):
        feat = list(model.feature_names_in_)
        missing = [f for f in feat if f not in Xc.columns]
        if missing:
            raise ValueError(f"Faltan columnas requeridas: {missing}")
        return Xc[feat]
    elif hasattr(model, "n_features_in_"):
        if model.n_features_in_ != Xc.shape[1]:
            print(f"WARNING: modelo espera {model.n_features_in_} features, X tiene {Xc.shape[1]}")
        return Xc
    else:
        return Xc

def evaluate_preds(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2}

# --- Evaluación ---
summary_rows = []

for model_key, fname in model_files.items():
    path = os.path.join(MODEL_DIR, fname)
    if not os.path.exists(path):
        print(f"Modelo no encontrado: {path} -> saltando")
        continue

    model = joblib.load(path)

    if "_A_" in model_key or model_key.endswith("_A_std"):
        X_train, X_test, y_train, y_test = X_A_train, X_A_test, y_A_train, y_A_test
        dataset_label = "A"
    elif "_B_" in model_key or model_key.endswith("_B_std"):
        X_train, X_test, y_train, y_test = X_B_train, X_B_test, y_B_train, y_B_test
        dataset_label = "B"
    else:
        X_train, X_test, y_train, y_test = X_A_train, X_A_test, y_A_train, y_A_test
        dataset_label = "A"

    X_train_al = align_features_for_model(X_train, model)
    X_test_al  = align_features_for_model(X_test, model)

    y_pred_train = model.predict(X_train_al)
    y_pred_test  = model.predict(X_test_al)

    # Asegurar que las predicciones sean 1D
    y_pred_train = np.ravel(y_pred_train)
    y_pred_test = np.ravel(y_pred_test)

    metrics_train = evaluate_preds(y_train, y_pred_train)
    metrics_test  = evaluate_preds(y_test, y_pred_test)

    gen_gap_rmse = metrics_test["RMSE"] - metrics_train["RMSE"]
    rel_gap = gen_gap_rmse / (metrics_train["RMSE"] + 1e-12)

    summary_rows.append({
        "model": model_key,
        "dataset": dataset_label,
        "MAE_train": metrics_train["MAE"],
        "MAE_test": metrics_test["MAE"],
        "RMSE_train": metrics_train["RMSE"],
        "RMSE_test": metrics_test["RMSE"],
        "R2_train": metrics_train["R2"],
        "R2_test": metrics_test["R2"],
        "gen_gap_rmse": gen_gap_rmse,
        "rel_gap_rmse": rel_gap
    })

    # --- Gráficos ---
    plt.figure(figsize=(6,5))
    plt.scatter(y_test, y_pred_test, alpha=0.6)
    mn, mx = min(y_test.min(), y_pred_test.min()), max(y_test.max(), y_pred_test.max())
    plt.plot([mn,mx], [mn,mx], linestyle="--")
    plt.title(f"Paridad TEST: {model_key}")
    plt.xlabel("y_true")
    plt.ylabel("y_pred")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"{model_key}_parity_test.png"))
    plt.close()

    resid = y_test - y_pred_test
    plt.figure(figsize=(6,4))
    plt.scatter(y_pred_test, resid, alpha=0.6)
    plt.axhline(0, linestyle="--")
    plt.title(f"Residuos TEST: {model_key}")
    plt.xlabel("y_pred")
    plt.ylabel("residuo (y_true - y_pred)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"{model_key}_residuals_test.png"))
    plt.close()

# --- Resumen final ---
summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values("RMSE_test")
summary_df.to_csv(os.path.join(OUTPUT_DIR, "model_evaluation_summary.csv"), index=False)
print(summary_df)


                       model dataset  MAE_train  MAE_test  RMSE_train  \
4              XGBoost_A_std       A   0.163221  4.245537    0.240138   
2         RandomForest_B_std       B   1.717307  4.727157    2.580707   
0                LASSO_A_std       A   6.527099  7.213173    8.967902   
3  Regresor_Polinomico_A_std       A   6.546484  7.359551    8.904705   
1       Neural_Network_A_std       A   7.927870  7.699104   10.055259   

   RMSE_test  R2_train   R2_test  gen_gap_rmse  rel_gap_rmse  
4   6.314029  0.999529  0.709039      6.073892     25.293386  
2   6.957816  0.945576  0.646680      4.377109      1.696089  
0  10.391036  0.342800  0.211976      1.423134      0.158692  
3  10.470810  0.352030  0.199830      1.566105      0.175874  
1  10.580532  0.173768  0.182972      0.525273      0.052239  
